In [ ]:
print("---------preparing training data for type_label features")

In [1]:
import sys,json,re,string
from typing import Callable, Dict, List, Set, Tuple
from elasticsearch import Elasticsearch
import gensim.downloader as api
import gensim
import datetime
import csv
sys.path.insert(1, 'extract_features')
sys.path.insert(1, 'util')
from helper_function import preprocess
from extract_No1to5_features import extract_features_1to5
from extract_No11to12_features import extract_features_11to12
from extract_No13to15_features import TypeTaxonomy, extract_features_13to15
from extract_No16_feature import extract_features_16
from extract_No17to19_features import get_analyze,extract_features_17to19
from extract_No20to22_features import extract_features_20to22
from extract_No23to25_features_optimized import extract_features_23to25

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Reduce logging level.
import logging
logging.basicConfig(level=logging.ERROR)

logging.getLogger("elasticsearch").disabled = True
#set a long timeout
es= Elasticsearch(timeout=600)
es.info()

{'name': 'LAPTOP-ADBLIUPR',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '5VELmBwJTk-urTuhZdTgew',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [4]:
!which python
!which pip

/c/Users/junec/anaconda3/python
/c/Users/junec/anaconda3/Scripts/pip


In [5]:
#load model
try:
    model_loaded = gensim.models.keyedvectors.KeyedVectors.load('googleNews.d2v')
except:
    model_loaded = api.load('word2vec-google-news-300')
    model_loaded.save('googleNews.d2v')
    model_loaded = gensim.models.keyedvectors.KeyedVectors.load('googleNews.d2v')

2021-11-20 06:41:46,913 : INFO : loading KeyedVectors object from googleNews.d2v
2021-11-20 06:41:48,353 : INFO : loading vectors from googleNews.d2v.vectors.npy with mmap=None
2021-11-20 06:42:08,299 : INFO : KeyedVectors lifecycle event {'fname': 'googleNews.d2v', 'datetime': '2021-11-20T06:42:08.267898', 'gensim': '4.1.2', 'python': '3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


In [6]:
#load files
filepath="data/DBpedia_map_type_entities.json"
with open(filepath, 'r', encoding='utf-8') as file:
    DBpedia_map_type_entities = json.load(file)
    
filepath="data/training_types.json"
with open(filepath,encoding='utf-8') as json_file:
    training_map_type_questions = json.load(json_file)

filepath="data/ElasticSearch_map_type_docID.json"
with open(filepath, 'r',encoding='utf-8') as f:
    docID_DBOtype_dict = json.load(f)

filepath="../smart-dataset/datasets/DBpedia/smarttask_dbpedia_train.json"
with open(filepath, 'r') as f:
    smarttask_dbpedia_train = json.load(f)

typeobj=TypeTaxonomy("data/dbpedia_types.tsv")

In [10]:
def extract_features_type_label(
    question:str,
    dp_type: str,
    training_map_type_questions:Dict,
    model_loaded:gensim.models.keyedvectors.KeyedVectors,
    es: Elasticsearch,
    analyze,X,terms_corpus,
    add_extra_features:bool=False
) -> List[float]:
    """Extracts features of a query and document pair.

        Args:
            query: string.
            dp_type: DBO type.
            es: Elasticsearch object instance.

        Returns:
            List of extracted feature values in a fixed order.
    """  
    features_17to19=extract_features_17to19(analyze,X,terms_corpus,training_map_type_questions,dp_type,question)
    feature_vect=list(features_17to19.values())
    
    features_20to22=extract_features_20to22(dp_type,question)
    feature_vect.extend(list(features_20to22.values()))
    
    features_23to25=extract_features_23to25(model_loaded,dp_type, question, mode="Euclidean")
    feature_vect.extend(list(features_23to25.values()))

    if add_extra_features:
        features_23to25_variant=extract_features_23to25(model_loaded,dp_type, question, mode="similarities")
        feature_vect.extend(list(features_23to25_variant.values()))
    
    return feature_vect

In [13]:
def prepare_training_data_type_label(  smarttask_dbpedia_train:List[Dict],
                                        training_map_type_questions:Dict[str,str],
                                        model_loaded:gensim.models.keyedvectors.KeyedVectors,
                                        es: Elasticsearch,
                                        add_extra_features:bool=False)-> Tuple[List[List[float]], List[int]]:
    """X and y label for type label features. Save to csv file.
    
         Args:
            smarttask_dbpedia_train: smarttask training set.
            training_map_type_questions:dictionary for DBOtype and all questions have that type in training dataset.
            model_loaded:pretrained gensim model.
            add_extra_features:if is true, will add an extra feature to X.
            es: Elasticsearch object instance.
    """

    analyze,X,terms_corpus=get_analyze(training_map_type_questions)
    
    count=0
    j=0

    with open("data/for_training_type_label.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)


        for entry in smarttask_dbpedia_train:
            if j%100==0:
                print("--------------------j:",j)

            j+=1
            if entry['question']==None:
                continue

            question_processed=preprocess(entry['question'])

            if entry['category']=='resource':
                if count%30==0:
                    print("-----------------",datetime.datetime.now())
                    print("--------------------count:",count)  
#                   print(f'--------------length of X_train:{len(X_train)} y_train: {len(y_train)}')
                count+=1
                #print("question_processed:",question_processed)
                for DBOtype in entry['type']: 
                    try:
                        features=extract_features_type_label(question_processed,
                                                            DBOtype,
                                                            training_map_type_questions,
                                                            model_loaded,
                                                            es,
                                                            analyze,X,terms_corpus) 
                    except BaseException as err:
                        print("------------error for type:",DBOtype,entry['question'])
                        print(f"Unexpected {err}, {type(err)}")  
                        raise

                    writer.writerow(features)
                    writer.writerow([1])

                #deal with top 30 documents
                hits = es.search(
                    index="dbpdiea_type_centric", q=question_processed, _source=True, size=30
                )["hits"]["hits"]
                rank_list= [hit['_source']["type"] for hit in hits]
                #print(rank_list)

                for DBOtype in rank_list:
                    if DBOtype not in entry['type']:
                        try:
                            features=extract_features_type_label(question_processed,
                                                            DBOtype,
                                                            training_map_type_questions,
                                                            model_loaded,
                                                            es,
                                                            analyze,X,terms_corpus) 
                        except BaseException as err:
                            print("------------error for type:",DBOtype,entry['question'])
                            print(f"Unexpected {err}, {type(err)}")  
                            raise
                            
                        writer.writerow(features)
                        writer.writerow([0])

    print(f'{count} features has been saved,{j} questions have been processed')

    return None



prepare_training_data_type_label(smarttask_dbpedia_train,
                                        training_map_type_questions,
                                        model_loaded,
                                        es)

--------------------j: 0
----------------- 2021-11-20 06:42:12.250400
--------------------count: 0


C:\Users\junec\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\junec\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


----------------- 2021-11-20 06:45:28.287709
--------------------count: 30
--------------------j: 100
----------------- 2021-11-20 06:48:19.671073
--------------------count: 60
----------------- 2021-11-20 06:50:41.771298
--------------------count: 90
--------------------j: 200
----------------- 2021-11-20 06:52:39.138388
--------------------count: 120
----------------- 2021-11-20 06:55:40.045112
--------------------count: 150
--------------------j: 300
----------------- 2021-11-20 06:57:42.145476
--------------------count: 180
----------------- 2021-11-20 07:00:42.011795
--------------------count: 210
--------------------j: 400
----------------- 2021-11-20 07:03:02.496809
--------------------count: 240
----------------- 2021-11-20 07:05:32.707688
--------------------count: 270
--------------------j: 500
----------------- 2021-11-20 07:08:14.221916
--------------------count: 300
----------------- 2021-11-20 07:11:02.133023
--------------------count: 330
--------------------j: 600
-----

--------------------j: 5000
----------------- 2021-11-20 10:34:45.248544
--------------------count: 2730
----------------- 2021-11-20 10:36:58.418805
--------------------count: 2760
--------------------j: 5100
----------------- 2021-11-20 10:39:43.734570
--------------------count: 2790
----------------- 2021-11-20 10:42:25.007195
--------------------count: 2820
--------------------j: 5200
----------------- 2021-11-20 10:44:49.134152
--------------------count: 2850
--------------------j: 5300
----------------- 2021-11-20 10:47:14.321605
--------------------count: 2880
----------------- 2021-11-20 10:49:57.401339
--------------------count: 2910
--------------------j: 5400
----------------- 2021-11-20 10:52:11.113223
--------------------count: 2940
----------------- 2021-11-20 10:54:25.981928
--------------------count: 2970
--------------------j: 5500
----------------- 2021-11-20 10:57:11.950312
--------------------count: 3000
--------------------j: 5600
----------------- 2021-11-20 10:59

--------------------j: 10000
----------------- 2021-11-20 14:56:52.154936
--------------------count: 5400
----------------- 2021-11-20 14:59:56.863065
--------------------count: 5430
--------------------j: 10100
----------------- 2021-11-20 15:01:57.364930
--------------------count: 5460
----------------- 2021-11-20 15:04:40.795689
--------------------count: 5490
--------------------j: 10200
----------------- 2021-11-20 15:07:13.668081
--------------------count: 5520
----------------- 2021-11-20 15:09:46.721528
--------------------count: 5550
--------------------j: 10300
----------------- 2021-11-20 15:12:19.247460
--------------------count: 5580
--------------------j: 10400
----------------- 2021-11-20 15:15:19.153944
--------------------count: 5610
----------------- 2021-11-20 15:18:24.703624
--------------------count: 5640
--------------------j: 10500
----------------- 2021-11-20 15:20:57.832247
--------------------count: 5670
----------------- 2021-11-20 15:24:27.350173
-----------

--------------------j: 14800
----------------- 2021-11-20 20:15:53.978400
--------------------count: 8070
----------------- 2021-11-20 20:19:06.619841
--------------------count: 8100
--------------------j: 14900
----------------- 2021-11-20 20:21:54.217794
--------------------count: 8130
----------------- 2021-11-20 20:25:11.391996
--------------------count: 8160
--------------------j: 15000
----------------- 2021-11-20 20:28:43.692868
--------------------count: 8190
----------------- 2021-11-20 20:31:59.831169
--------------------count: 8220
--------------------j: 15100
----------------- 2021-11-20 20:34:59.053905
--------------------count: 8250
----------------- 2021-11-20 20:37:51.835839
--------------------count: 8280
--------------------j: 15200
----------------- 2021-11-20 20:40:59.422644
--------------------count: 8310
----------------- 2021-11-20 20:44:28.645908
--------------------count: 8340
--------------------j: 15300
----------------- 2021-11-20 20:47:31.398514
-----------

the last line of above output is not correct, should be "9573 questions with the resource category are processed"